In [1]:
import pandas as pd
df_sber = pd.read_csv('Data/moex-SBER-170523-090423.csv', 
                      delimiter = ";", decimal = ',', encoding = 'latin-1')

df_lkoh = pd.read_csv('Data/moex-LKOH-170523-090423.csv', 
                      delimiter = ";", decimal = ',',encoding = 'latin-1')

In [2]:
#данные предоставлены из доски на 9 апреля 2023 года

sber_call_option_data = df_sber[[
    'CALL: Òåîðåòè÷åñêàÿ öåíà', 
    'ÑÒÐÀÉÊ', 
    'IV']].rename(columns = {
                             'CALL: Òåîðåòè÷åñêàÿ öåíà': 'Price',
                             'ÑÒÐÀÉÊ' : 'Strike'})
sber_call_option_data['IV'] /= 100

lkoh_call_option_data = df_lkoh[[
    'CALL: Òåîðåòè÷åñêàÿ öåíà', 
    'ÑÒÐÀÉÊ', 
    'IV']].rename(columns = {
                             'CALL: Òåîðåòè÷åñêàÿ öåíà': 'Price',
                             'ÑÒÐÀÉÊ' : 'Strike'})
lkoh_call_option_data['IV'] /= 100

In [3]:
sber_call_option_data.to_csv('Data/SBER_clear_170523_090423.csv', index=False)
lkoh_call_option_data.to_csv('Data/LKOH_clear_170523_090423.csv', index=False)

In [4]:
df_aapl = pd.read_excel('Data/nsdq-AAPL-07072023-27052023.xlsx')
df_amzn = pd.read_excel('Data/nsdq-AMZN-07072023-27052023.xlsx')

In [5]:
aapl_call_option_data = df_aapl[['Ask', 'Strike']].rename(columns = {'Ask': 'Price'})
amzn_call_option_data = df_amzn[['Ask', 'Strike']].rename(columns = {'Ask': 'Price'})

In [10]:
import numpy as np
from scipy.stats import norm
# NASDAQ не публикует подразумеваемую волатильность, поэтому вычисляем ее по формуле Блэка-Шоулза

r = 5.15371 / 100
T = 41/365
F0_aapl = 175.43
F0_amzn = 120.11

def bs_call(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    V = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return V

def vega(S0, K, T, r, sigma):
    d1 = ((np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))
    vega = S0 * norm.cdf(d1, 0.0, 1.0) * np.sqrt(T)
    return vega

def implied_vol(S0, K, T, r, C0):
    sigma_est = 1
    it = 100
    for i in range(it):
        sigma_est -= (bs_call(S0, K, T, r, sigma_est) - C0) / vega(S0, K, T, r, sigma_est)
    return sigma_est

In [11]:
aapl_call_option_data['IV'] = implied_vol(
            C0 = aapl_call_option_data['Price'],
            S0 = F0_aapl, 
            K = aapl_call_option_data['Strike'],
            T = T,
            r = r)

amzn_call_option_data['IV'] = implied_vol(
            C0 = amzn_call_option_data['Price'],
            S0 = F0_amzn, 
            K = amzn_call_option_data['Strike'],
            T = T,
            r = r)

In [12]:
aapl_call_option_data.to_csv('Data/AAPL_clear_07072023_27052023.csv', index=False)
amzn_call_option_data.to_csv('Data/AMZN_clear_07072023_27052023.csv', index=False)